Construction of the graph

We can define a graph over the CIFAR10 dataset as follows: each image is a node, and two nodes
form an edge if their are similar to each other. You can use any measure of similarity between
images, such as L2 distance on VGG or ResNet features (link), or LPIPS.
Contruct your graph by first computing a pairwise similarity matrix, converting it to an adjacency
matrix, and then visualizing the graph using NetworkX (link).
Note that you should build the graph with all the CIFAR10 data including both the training
and testing data.

In [1]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import numpy as np
from scipy.spatial.distance import cdist
import networkx as nx
import matplotlib.pyplot as plt

# Load CIFAR10 dataset
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
cifar10_data = torch.utils.data.ConcatDataset([cifar10_train, cifar10_test])

# Select a 10% random subset of the dataset
total_size = len(cifar10_data)
subset_size = int(0.1 * total_size)
indices = np.random.choice(range(total_size), subset_size, replace=False)
cifar10_subset = Subset(cifar10_data, indices)

# DataLoader for batch processing
batch_size = 100
data_loader = DataLoader(cifar10_subset, batch_size=batch_size, shuffle=False)

# Load a pre-trained ResNet model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True).to(device).eval()

# Extract features
def extract_features(data_loader, model, device):
    features = []
    with torch.no_grad():
        for batch, _ in data_loader:
            batch = batch.to(device)
            batch_features = model(batch)
            features.append(batch_features.cpu().numpy())
    return np.concatenate(features)

features = extract_features(data_loader, model, device)

# Compute L2 distances and create adjacency matrix
l2_distances = cdist(features, features, 'euclidean')
threshold = 50  # Adjust threshold based on your requirement
#print(l2_distances)
adjacency_matrix = (l2_distances < threshold).astype(int)

G = nx.Graph()
for i in range(len(adjacency_matrix)):
    for j in range(i + 1, len(adjacency_matrix)):
        if adjacency_matrix[i][j] == 1:
            G.add_edge(i, j)

plt.figure(figsize=(12, 12))
nx.draw_networkx(G, with_labels=False, node_size=10, node_color='red')
plt.show()
#
# Save the feature matrix, adjacency matrix, and labels
torch.save(features, 'features.pt')
torch.save(adjacency_matrix, 'adjacency_matrix.pt')
labels = cifar10_train.targets + cifar10_test.targets
labels_subset = [labels[i] for i in indices]
torch.save(labels_subset, 'labels.pt')


/home/myid/md40040/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'scipy'

3.2 Learning on the graph (6 pt)

CIFAR10 contains 10 classes. While we can train a neural network model to predict the classes
of an image using its image feature directly, let’s try it on the graph we just constructed.
To learn on the graph, you can use the pytorch geometry library. A quick start guide is here.
Construct a graph network of your choice, predict each node/image’s class using both its own
features and information from its neighbors. Train your GNN with training images and their
labels. What is the best performance you can achieve on the test set using a GNN? Note for a
simple CNN model, CIFAR10 test accuracy is about 70% or 80%.

In [15]:

import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, global_mean_pool
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR


features = torch.load('features.pt')
adjacency_matrix = torch.load('adjacency_matrix.pt')
labels = torch.load('labels.pt')

# Convert features and labels to tensors
features = torch.tensor(features, dtype=torch.float)
labels = torch.tensor(labels, dtype=torch.long)

# Convert adjacency matrix to edge index
edge_index = torch.tensor(np.array(np.nonzero(adjacency_matrix)), dtype=torch.long)

# Split data into training and testing sets
train_indices, test_indices = train_test_split(np.arange(len(labels)), test_size=0.1, random_state=42)
train_mask = torch.zeros(len(labels), dtype=torch.bool).index_fill_(0, torch.tensor(train_indices), True)
test_mask = torch.zeros(len(labels), dtype=torch.bool).index_fill_(0, torch.tensor(test_indices), True)

# Create PyTorch Geometric data
data = Data(x=features, edge_index=edge_index, y=labels)
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(data.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.out = torch.nn.Linear(hidden_channels, 10)  # 10 classes

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)  # No dropout before final layer

        return self.out(x)  # Directly return the output


# Initialize model and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model and optimizer
model = GNN(hidden_channels=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
scheduler = StepLR(optimizer, step_size=50, gamma=0.5)

data = data.to(device)

# Define training function
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.cross_entropy(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()
    return loss


# Define testing function
def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    test_correct = pred[test_mask] == data.y[test_mask]
    test_acc = int(test_correct.sum()) / int(test_mask.sum())
    return test_acc



# Modified training loop with scheduler
for epoch in range(400):  # Increased epochs
    loss = train()
    scheduler.step()
    print(f'Epoch {epoch}, Loss: {loss:.4f}')

# Evaluate the model
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')
data.batch = torch.zeros(data.num_nodes, dtype=torch.long, device=device)  # Dummy batch indices


Epoch 0, Loss: 2.6008
Epoch 1, Loss: 3.2865
Epoch 2, Loss: 3.0829
Epoch 3, Loss: 2.6650
Epoch 4, Loss: 2.4929
Epoch 5, Loss: 2.4454
Epoch 6, Loss: 2.3674
Epoch 7, Loss: 2.2426
Epoch 8, Loss: 2.2661
Epoch 9, Loss: 2.1810
Epoch 10, Loss: 2.1345
Epoch 11, Loss: 2.1027
Epoch 12, Loss: 2.0485
Epoch 13, Loss: 2.0299
Epoch 14, Loss: 2.0104
Epoch 15, Loss: 1.9935
Epoch 16, Loss: 1.9939
Epoch 17, Loss: 1.9883
Epoch 18, Loss: 1.9686
Epoch 19, Loss: 1.9518
Epoch 20, Loss: 1.9462
Epoch 21, Loss: 1.9583
Epoch 22, Loss: 1.9598
Epoch 23, Loss: 1.9476
Epoch 24, Loss: 1.9448
Epoch 25, Loss: 1.9365
Epoch 26, Loss: 1.9269
Epoch 27, Loss: 1.9232
Epoch 28, Loss: 1.9185
Epoch 29, Loss: 1.9194
Epoch 30, Loss: 1.9136
Epoch 31, Loss: 1.9071
Epoch 32, Loss: 1.9021
Epoch 33, Loss: 1.9070
Epoch 34, Loss: 1.8984
Epoch 35, Loss: 1.8990
Epoch 36, Loss: 1.8916
Epoch 37, Loss: 1.8833
Epoch 38, Loss: 1.8826
Epoch 39, Loss: 1.8741
Epoch 40, Loss: 1.8773
Epoch 41, Loss: 1.8612
Epoch 42, Loss: 1.8699
Epoch 43, Loss: 1.850

In [14]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GraphConv
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau


# Load the saved features, adjacency matrix, and labels
features = torch.load('features.pt')
adjacency_matrix = torch.load('adjacency_matrix.pt')
labels = torch.load('labels.pt')

# Convert features and labels to tensors
features = torch.tensor(features, dtype=torch.float)
labels = torch.tensor(labels, dtype=torch.long)

# Convert adjacency matrix to edge index
edge_index = torch.tensor(np.array(np.nonzero(adjacency_matrix)), dtype=torch.long)

# Split data into training and testing sets
train_indices, test_indices = train_test_split(np.arange(len(labels)), test_size=0.2, random_state=42)
train_mask = torch.zeros(len(labels), dtype=torch.bool).index_fill_(0, torch.tensor(train_indices), True)
test_mask = torch.zeros(len(labels), dtype=torch.bool).index_fill_(0, torch.tensor(test_indices), True)

# Create PyTorch Geometric data
data = Data(x=features, edge_index=edge_index, y=labels)

# Define a simpler GNN model
class SimpleGNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(SimpleGNN, self).__init__()
        self.conv1 = GCNConv(data.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 10)  # Output size = Number of classes

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize model and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
# Initialize model and optimizer
model = SimpleGNN(hidden_channels=32).to(device)  # Reduced hidden channels
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.7, patience=10, min_lr=0.00001)


data = data.to(device)
# Modify the training loop to include evaluation
def train(epoch):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.cross_entropy(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()
    return loss

def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    correct = pred[test_mask] == data.y[test_mask]
    acc = int(correct.sum()) / int(test_mask.sum())
    return acc

# Training loop
best_acc = 0
for epoch in range(200):
    loss = train(epoch)
    acc = test()
    scheduler.step(acc)
    if acc > best_acc:
        best_acc = acc
    print(f'Epoch {epoch}, Loss: {loss:.4f}, Test Acc: {acc:.4f}')

print(f'Best Test Accuracy: {best_acc:.4f}')

Epoch 0, Loss: 3.9956, Test Acc: 0.1017
Epoch 1, Loss: 12.0141, Test Acc: 0.1042
Epoch 2, Loss: 11.9685, Test Acc: 0.0967
Epoch 3, Loss: 10.4229, Test Acc: 0.1275
Epoch 4, Loss: 9.6707, Test Acc: 0.1650
Epoch 5, Loss: 8.1464, Test Acc: 0.0942
Epoch 6, Loss: 5.9559, Test Acc: 0.0950
Epoch 7, Loss: 4.2876, Test Acc: 0.1075
Epoch 8, Loss: 2.6433, Test Acc: 0.1308
Epoch 9, Loss: 2.2780, Test Acc: 0.1425
Epoch 10, Loss: 2.2540, Test Acc: 0.1225
Epoch 11, Loss: 2.2278, Test Acc: 0.1100
Epoch 12, Loss: 2.2277, Test Acc: 0.1083
Epoch 13, Loss: 2.2333, Test Acc: 0.1083
Epoch 14, Loss: 2.2390, Test Acc: 0.1775
Epoch 15, Loss: 2.2432, Test Acc: 0.1533
Epoch 16, Loss: 2.2451, Test Acc: 0.1833
Epoch 17, Loss: 2.2447, Test Acc: 0.1833
Epoch 18, Loss: 2.2423, Test Acc: 0.1833
Epoch 19, Loss: 2.2381, Test Acc: 0.1833
Epoch 20, Loss: 2.2328, Test Acc: 0.1825
Epoch 21, Loss: 2.2271, Test Acc: 0.1525
Epoch 22, Loss: 2.2231, Test Acc: 0.1817
Epoch 23, Loss: 2.2179, Test Acc: 0.1800
Epoch 24, Loss: 2.2155,

3.3 Subgraph performance (5 pt Extra Credit)

Can you find a sub-graph (i.e., a subset number of nodes and their connections) from the test
set that has the similar performance of the whole test set? In other words, can we leverage the
graph structure to find a smaller number of samples to give us a good estimation of the overall
performance of a model on our data?

In [38]:
import torch
import networkx as nx
import numpy as np

# Load the model, features, edge_index, and labels as before
model = GNN(hidden_channels=64).to(device)
# Load the saved features, adjacency matrix, and labels
features = torch.load('features.pt')
adjacency_matrix = torch.load('adjacency_matrix.pt')
labels = torch.load('labels.pt')

# Convert features and labels to tensors
features = torch.tensor(features, dtype=torch.float)
labels = torch.tensor(labels, dtype=torch.long)

# Convert adjacency matrix to edge index
edge_index = torch.tensor(np.array(np.nonzero(adjacency_matrix)), dtype=torch.long)

features = torch.tensor(features, dtype=torch.float).to(device)
labels = torch.tensor(labels, dtype=torch.long).to(device)
edge_index = torch.tensor(np.array(np.nonzero(adjacency_matrix)), dtype=torch.long).to(device)

# Convert edge_index to a NetworkX graph for centrality calculation
G = nx.Graph()
edge_index_np = edge_index.cpu().numpy()
G.add_edges_from(zip(edge_index_np[0], edge_index_np[1]))


# Calculate degree centrality
degree_centrality = nx.degree_centrality(G)
# Sort nodes by centrality and pick top nodes for the subgraph
sorted_nodes = sorted(degree_centrality, key=degree_centrality.get, reverse=True)
top_central_nodes = sorted_nodes[:600]  # Select top nodes, e.g., 600

# Convert to mask
subgraph_mask = torch.zeros(len(labels), dtype=torch.bool)
subgraph_mask[top_central_nodes] = True
subgraph_test_mask = subgraph_mask & test_mask

# Test model on the subgraph
def test_subgraph(mask):
    model.eval()
    with torch.no_grad():
        out = model(data)
        pred = out.argmax(dim=1)
        correct = pred[mask] == data.y[mask]
        acc = int(correct.sum()) / int(mask.sum())
    return acc

while (subgraph_accuracy< 0.41):
    subgraph_accuracy = test_subgraph(subgraph_test_mask)
    print(f'Subgraph Accuracy: {subgraph_accuracy:.4f}')


/tmp/ipykernel_51138/1729079141.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(features, dtype=torch.float).to(device)
/tmp/ipykernel_51138/1729079141.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)


Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Accuracy: 0.0606
Subgraph Acc

KeyboardInterrupt: 